# Download and Extract Data


In [5]:
mkdir data/

In [6]:
!wget https://cernbox.cern.ch/index.php/s/ZUHveJKajnZNwTA/download -O data/ZUHveJKajnZNwTA.tar

--2022-08-07 02:21:41--  https://cernbox.cern.ch/index.php/s/ZUHveJKajnZNwTA/download
Resolving cernbox.cern.ch (cernbox.cern.ch)... 128.142.53.35, 137.138.120.151, 128.142.53.28, ...
Connecting to cernbox.cern.ch (cernbox.cern.ch)|128.142.53.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘data/ZUHveJKajnZNwTA.tar’

data/ZUHveJKajnZNwT     [        <=>         ]   4.51G  81.3MB/s    in 61s     

2022-08-07 02:22:43 (76.2 MB/s) - ‘data/ZUHveJKajnZNwTA.tar’ saved [4848198144]



In [7]:
cd /content/data

/content/data


In [8]:
import tarfile
my_tar = tarfile.open('ZUHveJKajnZNwTA.tar')
my_tar.extractall('./data') # specify which folder to extract to
my_tar.close()

ReadError: ignored

In [ ]:
cd /content/data

# Convert Data Into Images


In [ ]:
cd /content/data/data

/content/data/data


In [ ]:
import numpy as np
import pyarrow.parquet as pq
import os
import time as time
import PIL.Image as imm
path = "/content/data"
import cv2

In [ ]:
import os
os.makedirs("./Images/0")
os.makedirs("./Images/1")

In [ ]:
def generate(pf):
    record_batch = pf.iter_batches(batch_size=1024)
    while(True):
        try:
            batch = next(record_batch)
            print(batch.num_rows)
            transform(batch)
        except StopIteration:
            print("Done")
            break

In [ ]:
alpha = 0
beta =  0
def saver(im,meta):
    global alpha
    global beta
    for i in range(meta.shape[0]):
        img = im[i,:,:,:]
        img = img.T
        img[:,:,0] = ( img[:,:,0] - img[:,:,0].min() ) / ( img[:,:,0].max()-img[:,:,0].min() )
        img[:,:,1] = ( img[:,:,1] - img[:,:,1].min() ) / ( img[:,:,1].max()-img[:,:,1].min() )
        img[:,:,2] = ( img[:,:,2] - img[:,:,2].min() ) / ( img[:,:,2].max()-img[:,:,2].min() )
        img = img*255
        img = img.astype(np.uint8)
        #lmm = imm.fromarray(img)
        if(meta[i] == 0):
            impath = path+"1/"+str(alpha)+".png"
            alpha = alpha + 1
        if(meta[i] == 1):
            impath = path+"0/"+str(beta)+".png"
            beta = beta + 1
        #lmm.save(impath)
        cv2.imwrite(impath , img)

In [ ]:
def transform(batch):
    p = batch.to_pandas()
    im = np.array(np.array(np.array(p.iloc[:,0].tolist()).tolist()).tolist())
    meta = np.array(p.iloc[:,3])
    saver(im,meta)

In [ ]:
generate(pq.ParquetFile("QCDToGGQQ_IMGjet_RH1all_jet0_run0_n36272.test.snappy.parquet"))
generate(pq.ParquetFile("QCDToGGQQ_IMGjet_RH1all_jet0_run1_n47540.test.snappy.parquet"))
generate(pq.ParquetFile("QCDToGGQQ_IMGjet_RH1all_jet0_run2_n55494.test.snappy.parquet"))

1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
432
Done
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
436
Done
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
198
Done


# Preprocessing (Pytorch)

In [ ]:
import tensorflow as tf
import numpy as np
np.random.seed(0)
import os, glob
import time
import h5py
import pyarrow as pa
import pyarrow.parquet as pq
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import ConcatDataset, Dataset, DataLoader, sampler, DistributedSampler
from sklearn.metrics import roc_curve, auc
from torch.utils.data import *
import numpy
import time
import h5py
from keras.models import Sequential
from keras.initializers import TruncatedNormal
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.callbacks import ReduceLROnPlateau
import math 

from sklearn.metrics import roc_curve, auc

import matplotlib.pyplot as plt

In [ ]:
class ParquetDataset(Dataset):
    def __init__(self, filename, transform=None):
        self.parquet = pq.ParquetFile(filename)
        self.cols = None 
        self.transform = transform
    def __getitem__(self, index):
        data = self.parquet.read_row_group(index, columns=self.cols).to_pydict()
        # Preprocessing
        start = time.time()
        data['X_jets'] = np.float32(data['X_jets'])
        data['X_jets'] = torch.Tensor(data['X_jets'])
        data['y'] = torch.Tensor(data['y'])
        
        #to preprocess
        data['X_jets'][data['X_jets'] < 1.e-3] = 0. # Zero-Suppression
        data['X_jets'][-1,...] = 25.*data['X_jets'][-1,...] # For HCAL: to match pixel intensity distn of other layers
        data['X_jets'] = data['X_jets']/100. # To standardize
        #data['X'] = np.float32(data['X'][0])
        #print("List lenth",data['X_jets'])
        #data['X_jets'] = numpy.array(data['X_jets'])
        
        if self.transform:
            #data['X_jets'] = numpy.moveaxis(data['X_jets'], 1,3)
            #print("Trnsforn : ",data['X_jets'].shape)
            data['X_jets']= self.transform(data['X_jets'][0]) 
        #data['X_jets'] = numpy.moveaxis(data['X_jets'], -1,1)
        
        #print("Lter : ",data['X_jets'].shape)
        
        end = time.time()
        #print("Time required for preprocessing : ",end - start)
        
        #print("Mean : ", torch.mean(data['X_jets']))
        return dict(data)
    def __len__(self):
        return self.parquet.num_row_groups

In [ ]:
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
def get_data_loader(datasets, batch_size, cut, random_sampler=False):

    transform = transforms.Compose([
      transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
      transforms.RandomHorizontalFlip(),
      transforms.RandomRotation(20)
    ])
    #scripted_transforms = torch.jit.script(transforms)
  
    dset = ConcatDataset([ParquetDataset(dataset, transform=transform) for dataset in datasets])
    idxs = np.random.permutation(len(dset))
    if random_sampler: 
        random_sampler = sampler.SubsetRandomSampler(idxs[:cut])
    else: 
        random_sampler = None 
    validation_split = .2
    shuffle_dataset = True
    random_seed= 42

# Creating data indices for training and validation splits:
    dataset_size = 55494
    indices = list(range(dataset_size))
    split = int(np.floor(validation_split * dataset_size))
    print(split)
    if shuffle_dataset :
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(val_indices)
    print("Train indices ",train_sampler)
    print("Test indices ", valid_sampler)
    train_loader = torch.utils.data.DataLoader(dset, batch_size=batch_size, 
                                           sampler=train_sampler)
    test_loader = torch.utils.data.DataLoader(dset, batch_size=batch_size,
                                                sampler=valid_sampler)
    
    #data_loader = DataLoader(dataset=dset, batch_size=batch_size, shuffle=False, num_workers=10, sampler=random_sampler, pin_memory=True)
    return train_loader, test_loader

In [ ]:
datasets_train = ['QCDToGGQQ_IMGjet_RH1all_jet0_run0_n215556.train.snappy.parquet', 
                  'QCDToGGQQ_IMGjet_RH1all_jet0_run1_n297980.train.snappy.parquet', 
                  'QCDToGGQQ_IMGjet_RH1all_jet0_run2_n280364.train.snappy.parquet']
datasets_test = ['QCDToGGQQ_IMGjet_RH1all_jet0_run0_n36272.test.snappy.parquet',
                 'QCDToGGQQ_IMGjet_RH1all_jet0_run1_n47540.test.snappy.parquet',
                 'QCDToGGQQ_IMGjet_RH1all_jet0_run2_n55494.test.snappy.parquet']

In [ ]:
cd /content/data/data

In [ ]:
train_loader, test_loader= get_data_loader(datasets_train, 32, cut = None, random_sampler = True)
#test_loader = get_data_loader(datasets_test, 32, cut = None, random_sampler = True)

In [ ]:
for i, batch in enumerate(train_loader):
    X_jets,	pt,	m0,	y = batch["X_jets"], batch["pt"], batch["m0"],batch["y"]
    #print(torch.max(X_jets))
    if (i%50 ==0):
        print(torch.max(X_jets))

In [ ]:
def my_collate(batch):
    data = torch.cat([item[0] for item in batch],dim = 0)
    target = torch.cat([item[1] for item in batch],dim = 0)

    return [data, target]

In [ ]:
def train_epoch(model, optimizer, data_loader, loss_history):
    total_samples = len(data_loader)*32 #batch size =32
    correct_samples = 0
    model.train()
    
    for i, batch in enumerate(data_loader):
        optimizer.zero_grad()
        data = batch["X_jets"]
        data = data.cuda()
        output = model(data)
        target = batch["y"]
        target = target.cuda()
        target = torch.squeeze(target, dim=1)
        loss = nn.BCEWithLogitsLoss()(torch.squeeze(output, dim=1),target)
        pred = torch.squeeze(nn.Sigmoid()(output))
        pred[pred>=0.5] = 1
        pred[pred<0.5] = 0
        optimizer.step()
        
        correct_samples += pred.eq(target).sum()

        if i % 100 == 0:
            print('[' +  '{:5}'.format(i * len(data)) + '/' + '{:5}'.format(total_samples) +
                  ' (' + '{:3.0f}'.format(100 * i / len(data_loader)) + '%)]  Loss: ' +
                  '{:6.4f}'.format(loss.item()) + '  Accuracy:' + '{:5}'.format(correct_samples) + '/' +
                  '{:5}'.format(total_samples) + ' (' + '{:4.2f}'.format(100.0 * correct_samples / total_samples) + '%)')
            loss_history.append(loss.item())

In [ ]:
def evaluate(model, data_loader, loss_history):
    model.eval()
    
    total_samples = len(data_loader)*32 #len(data_loader.dataset)
    correct_samples = 0
    total_loss = 0

    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            
            data = batch["X_jets"]
            data = data.cuda()
            output = model(data)
            target = batch["y"]
            target = target.cuda()
            target = torch.squeeze(target, dim=1)
            loss = nn.BCEWithLogitsLoss()(torch.squeeze(output, dim=1),target)
            pred = torch.squeeze(nn.Sigmoid()(output))
            pred[pred>=0.5] = 1
            pred[pred<0.5] = 0
            total_loss += loss.item()
            correct_samples += pred.eq(target).sum()

    avg_loss = total_loss / (total_samples/32)
    loss_history.append(avg_loss) 
    print('\nAverage val loss: ' + '{:.4f}'.format(avg_loss) +
          '  Accuracy:' + '{:5}'.format(correct_samples) + '/' +
          '{:5}'.format(total_samples) + ' (' +
          '{:4.2f}'.format(100.0 * correct_samples / total_samples) + '%)\n')

# Model Selection, Training, and Evaluation (Pytorch)

In [ ]:
import torchvision.models as models
import torchvision
from torchvision.models import ResNet18_Weights
torchvision.models.ResNet18_Weights(ResNet18_Weights.DEFAULT)
#resnet18 = models.resnet18(weights='IMAGENET1K_V1')
resnet18 = models.resnet18(pretrained=True)

In [ ]:
N_EPOCHS = 8
import time
start_time = time.time()
from torch import nn
from sklearn.metrics import roc_curve, auc
model = resnet18
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
model.fc = nn.Sequential(nn.AdaptiveAvgPool1d(512),
                         nn.Dropout(0.3),
                         nn.Linear(512,256),
                         nn.ReLU(),
                         nn.Dropout(0.3),
                         nn.Linear(256,1),
                         nn.Sigmoid())
model.cuda()

cnt = 0
train_loss_history, test_loss_history = [], []
for epoch in range(1, N_EPOCHS + 1):
    
    r = 0.0005/(10**(cnt))
    print("LR:",r)
    optimizer = torch.optim.Adam(model.parameters(), lr=r)
    print('Epoch:', epoch)
    train_epoch(model, optimizer, train_loader, train_loss_history)
    print("Training done!")
    evaluate(model, test_loader, test_loss_history)
    pred_list, target_list = evaluate(model, test_loader, test_loss_history)
    fpr, tpr, _ = roc_curve(target_list, pred_list)
    roc_auc = auc(fpr, tpr)
    print('Training ROC AUC:', roc_auc)
    if epoch%4==0:
      cnt=cnt+1
    

print('Execution time:', '{:5.2f}'.format(time.time() - start_time), 'seconds')

In [ ]:
def test(model, data_loader, loss_history):
    model.eval()
    pred_list = []
    target_list = []
    total_samples = len(data_loader)*32
    correct_samples = 0
    total_loss = 0

    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            
            data = batch["X_jets"]
            data = data.cuda()
            output = model(data)
            target = batch["y"]
            target = target.cuda()          
            target = torch.squeeze(target, dim=1)
            loss = nn.BCEWithLogitsLoss()(torch.squeeze(output, dim=1),target)
            pred = torch.squeeze(nn.Sigmoid()(output))
            pred_list.extend(pred.tolist())
            target_list.extend(target.tolist())
            pred[pred>=0.5] = 1
            pred[pred<0.5] = 0
            total_loss += loss.item()
            correct_samples += pred.eq(target).sum()

    avg_loss = total_loss / (total_samples/32)
    loss_history.append(avg_loss)
    print('\nAverage test loss: ' + '{:.4f}'.format(avg_loss) +
          '  Accuracy:' + '{:5}'.format(correct_samples) + '/' +
          '{:5}'.format(total_samples) + ' (' +
          '{:4.2f}'.format(100.0 * correct_samples / total_samples) + '%)\n')
    return pred_list,target_list

In [ ]:
test_loss_history = []
pred_list, target_list = test(model, test_loader, test_loss_history)

In [ ]:
from sklearn.metrics import roc_curve, auc
fpr, tpr, _ = roc_curve(target_list, pred_list)
roc_auc = auc(fpr, tpr)
print('Test ROC AUC:', roc_auc)

In [ ]:
import matplotlib.pyplot as plt

plt.plot([0, 1], [0, 1], 'k--')
#plt.legend(loc=2, prop={'size': 15})
plt.plot(fpr, tpr, label='Model 1 (ROC-AUC = {:.3f})'.format(roc_auc))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve: Resnet 18 (modified FC Layer)')
plt.legend(loc='best')
plt.show()
plt.savefig('resnet_18_ROC_modified.png')


# Preprocessing (Keras)

In [ ]:
cd /content/data/data

In [ ]:
import numpy as np
import PIL.Image as im
import tensorflow as tf
import tensorflow.keras.layers as tfl
import matplotlib.pyplot as plt
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from keras.utils.vis_utils import plot_model

In [ ]:
train_datagen= ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.2
)
train_generator = train_datagen.flow_from_directory(
        '/kaggle/input/cmscommon2/Images',
        target_size= (125,125),
        color_mode='rgb',
        batch_size=32,
        class_mode='binary',
        subset='training',
        shuffle = True
)

validation_generator = train_datagen.flow_from_directory(
        '/kaggle/input/cmscommon2/Images',
        target_size= (125,125),
        color_mode='rgb',
        batch_size=32,
        class_mode='binary',
        subset='validation',
        shuffle = True

)